In [7]:
import os
import sys

# uncomment and replace with correct path if there are import errors
sys.path.append("/home/dux/surface_sampling/")
sys.path.append("/home/dux/NeuralForceField/")
os.environ["LAMMPS_POTENTIALS"] = "/home/jurgis/lammps/potentials/"
os.environ["LAMMPS_COMMAND"] ="/home/jurgis/lammps/src/lmp_serial"
os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

from mcmc import MCMC
from mcmc.slab import initialize_slab

from ase.calculators.lammpsrun import LAMMPS # ase version of LAMMPS

from time import perf_counter


Initialize test slab and parameters

In [8]:
# initialize some parameters first
# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
element = 'Cu'
slab = initialize_slab(Cu_alat, elem=element, size=(2, 2, 2), miller=(1, 0, 0))

surface_name = 'Cu(100)'
chem_pot = 0 # eV
alpha = 0.99 # slowly anneal
temp = 1.0 # in terms of kbT
num_sweeps = 50

/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(


Set up LAMMPS (calculator)

In [9]:
# use LAMMPS
optimizer = 'LAMMPS'
parameters = {
    'pair_style': 'eam',
    'pair_coeff': ['* * Cu_u3.eam']
}

# set up the LAMMPS calculator
potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'Cu_u3.eam')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=False, tmp_dir=os.path.join(os.path.expanduser('~'), "tmp_files"))
lammps_calc.set(**parameters)

{'pair_style': 'eam', 'pair_coeff': ['* * Cu_u3.eam']}

Perform MCMC and view results. Detailed results can be found in the corresponding run in the `Cu(100)/` folder.

In [10]:
mcmc = MCMC(calc=lammps_calc, canonical=False, testing=False, surface_name=surface_name, 
            element=element, adsorbates=element, 
            relax=False, optimizer=optimizer) # no relaxation

start = perf_counter()
# call the main function
mcmc.mcmc_run(num_sweeps=num_sweeps,
    temp=temp,
    pot=chem_pot,
    alpha=alpha,
    slab=slab
    )

stop = perf_counter()
print(f"Time taken = {stop - start} seconds")

2023-05-19 23:49:45,697|INFO|Running with num_sweeps = 50, temp = 1.0, pot = 0, alpha = 0.99
2023-05-19 23:49:45,698|INFO|using slab calc <ase.calculators.lammpsrun.LAMMPS object at 0x7fd6a03a9fa0>
2023-05-19 23:49:45,699|INFO|there are 8 atoms in pristine slab
2023-05-19 23:49:45,700|INFO|adsorbate(s) is(are) Cu
2023-05-19 23:49:45,820|INFO|In pristine slab, there are a total of 16 sites
2023-05-19 23:49:45,852|INFO|running for 16 iterations per run over a total of 50 runs
2023-05-19 23:49:45,854|INFO|In sweep 1 out of 50
2023-05-19 23:49:46,946|INFO|optim structure has Energy = -37.92087610044961
2023-05-19 23:49:46,949|INFO|In sweep 2 out of 50
2023-05-19 23:49:48,061|INFO|optim structure has Energy = -44.38191789040085
2023-05-19 23:49:48,072|INFO|In sweep 3 out of 50
2023-05-19 23:49:49,199|INFO|optim structure has Energy = -47.30809646549415
2023-05-19 23:49:49,216|INFO|In sweep 4 out of 50
2023-05-19 23:49:50,352|INFO|optim structure has Energy = -44.38191789040057
2023-05-19 23

KeyboardInterrupt: 